In [65]:
from main  import *
text = 'C:/Users/ASUS/AppData/Local/Temp/tmpa70zfck3.py'

vlayer = QgsVectorLayer(wards_shapefile, "airports", "ogr")
vlayer2 = QgsVectorLayer(lga_shapefile, "SHEET", "ogr")
vlayer3 = QgsVectorLayer(set_extent_shapefile, "SHEET", "ogr")


# for field in vlayer.fields():
#     print(field.name(), field.typeName())
# vlayer.displayField()

query = '"statename"  =  \'Kwara\' AND  "lganame"  =  \'Edu\'AND "wardname" = \'Tsonga 1\''

# print(len(list(vlayer.getFeatures())))
state = crt_subset_lyr(vlayer, query)
print(len(list(state.getFeatures())))
# state2  = export_selected(vlayer3,state)
# print((list(state.getFeatures())))

state_extent  = clip_set_ext_layer(vlayer3,state)
# # # clip_set_ext_layer(vlayer2,state)
print(list(state_extent.getFeatures()))

lon, lat = create_random_point(state_extent)
# # # clip_set_ext_layer(vlayer2,state)
print(lon, lat)

# processing.algorithmHelp("native:selectbylocation")
# parameters = { 'INPUT' : state_extent, 
#                'VALUE' : 1, 
#                 }
# point =processing.run('native:selectbylocation', parameters)
# /algs = dict()
# for alg in QgsApplication.processingRegistry().algorithms():
#     algs[alg.displayName()] = alg.id()
#     print(alg.displayName(), alg.id() )

1
{'OUTPUT': <QgsVectorLayer: 'Clipped' (memory)>}
[<qgis._core.QgsFeature object at 0x0000013F3B127D00>, <qgis._core.QgsFeature object at 0x0000013F467BB640>, <qgis._core.QgsFeature object at 0x0000013F467B8C10>, <qgis._core.QgsFeature object at 0x0000013F467B8AF0>, <qgis._core.QgsFeature object at 0x0000013F467B8B80>, <qgis._core.QgsFeature object at 0x0000013F467B8EE0>, <qgis._core.QgsFeature object at 0x0000013F467B88B0>, <qgis._core.QgsFeature object at 0x0000013F467B8DC0>, <qgis._core.QgsFeature object at 0x0000013F467B8F70>, <qgis._core.QgsFeature object at 0x0000013F467BB370>, <qgis._core.QgsFeature object at 0x0000013F467BB7F0>, <qgis._core.QgsFeature object at 0x0000013F467B9000>, <qgis._core.QgsFeature object at 0x0000013F467B9120>, <qgis._core.QgsFeature object at 0x0000013F467B91B0>, <qgis._core.QgsFeature object at 0x0000013F467B9240>, <qgis._core.QgsFeature object at 0x0000013F467B92D0>, <qgis._core.QgsFeature object at 0x0000013F467B9360>, <qgis._core.QgsFeature object 

In [76]:
def create_random_points(layer):   
    extents = [extent for extent in  layer.getFeatures() if extent["bld_count"] != "1-50"]
    print(extents)
    dct = {}
    for idx,extent in enumerate(extents):
        if extent["bld_count"]  ==  "51-100":
            dct[str(idx)] = 1
        elif extent["bld_count"]  ==  "51-100":
            dct[str(idx)] = 2
        else:
            dct[str(idx)] = 3
    # print(dct)
    # for extent in state_extent.getFeatures():
    #     print(extent["bld_count"], extent.geometry())
    extent = extents[int(max(dct.values()))]
    # print(extent["bld_count"])
    geom = extent.geometry() 
    random_x = random.uniform(geom.boundingBox().xMinimum(), geom.boundingBox().xMaximum())
    random_y = random.uniform(geom.boundingBox().yMinimum(), geom.boundingBox().yMaximum())
    return random_x, random_y

[<qgis._core.QgsFeature object at 0x0000013F47163250>, <qgis._core.QgsFeature object at 0x0000013F429F7F40>, <qgis._core.QgsFeature object at 0x0000013F47161AB0>, <qgis._core.QgsFeature object at 0x0000013F467B8A60>, <qgis._core.QgsFeature object at 0x0000013F467B8C10>, <qgis._core.QgsFeature object at 0x0000013F467B8DC0>, <qgis._core.QgsFeature object at 0x0000013F467BB520>, <qgis._core.QgsFeature object at 0x0000013F467B88B0>, <qgis._core.QgsFeature object at 0x0000013F467BB490>]
5.150918036400181 9.02154547934353


In [ ]:
# vlayer = load_vector_layer(wards_shapefile)
vlayer = QgsVectorLayer(wards_shapefile, "airports", "ogr")
vlayer2 = QgsVectorLayer(set_extent_shapefile, "SHEET", "ogr")

# for field in vlayer.fields():
#     print(field.name(), field.typeName())
# vlayer.displayField()
query = '"statename" = \'Kwara\' AND "lganame" = \'Edu\'AND "wardname" = \'Tsaragi 3\''
print(vlayer2.crs().authid())



# for ft in len(list(set_extent_layer.getFeatures())):

print(len(list(set_extent_layer.getFeatures())))
# print(len(list(vlayer2.getFeatures())))

In [3]:
def crt_subset_lyr(layer,query):
    # memory_layer = QgsVectorLayer(
    #     layer.dataProvider().dataSourceUri(),
    #     layer.name() + '_selected',
    #     'memory'
    # )
    set_extent_uri = f"polygon?crs={vlayer2.crs().authid()}"
    memory_layer = QgsVectorLayer(set_extent_uri, "set_extent", "memory")
    memory_layer.startEditing()
    memory_layer.dataProvider().addAttributes(layer.fields())
    memory_layer.updateFields()
    for field in memory_layer.fields()
    selected = vlayer2.getFeatures(QgsFeatureRequest().setFilterExpression(query))
    # selected_features = [feature for feature in selected]
    print(list(selected))
    # Create a memory layer
    

    # Set the memory layer's fields
   

    # Add the selected features to the memory layer
    print(list(selected))
    for feature in selected:
        memory_layer.addFeatures(feature)
    memory_layer.commitChanges()
    memory_layer.updateExtents()

    print(list(memory_layer.getFeatures()))
    return memory_layer

['1. ASA', '2. BARUTEN', '3. EDU', '4. EKITI', '5. IFELODUN', '6. ILORIN EAST', '7. ILORIN SOUTH ', '8. ILORIN WEST', '9. IREPODUN', '10. ISIN', '11. KAIAMA', '12. MORO', '13. OFFA', '14. OKE ERO', '15. OYUN', '16. PATIGI']


In [ ]:


for sheet in sheets[:2]:
    demo_df = pd.read_excel(file,sheet_name=sheet)
    row_index=actual_header_row(demo_df)
    clean_data = remove_first_blank_column(file,row_index,sheet)
    new_data = remove_unwanted_columns(clean_data,["Wards","List of contiguous communities/ settlements","Population\n(2023)"])
    base_data = remove_blank_wards_rows(new_data)
    wards_df = drop_subtotal_rows(base_data)
    # print(base_data["Wards"])
    final_df=create_random_cluster(wards_df) 

    print(final_df)
    

               Wards  Total communities  Population  Cumulative frequency  \
0  Okeweru Yowere II               66.0      9801.0               19091.0   
1       Owode/Gbogun               47.0     11430.0               51970.0   
2      Efue/Berikodo               53.0     13236.0               89813.0   
3      Onire/Odegiwa               72.0     14943.0              118230.0   
4          Budo Egba               36.0     15385.0              148725.0   
5              Ogele               53.0     16235.0              164960.0   
6             Laduba               17.0     19516.0              201447.0   
7        Otte/Ballah               41.0     31735.0              233182.0   

             Clusters  
0   Cluster 1 (18668)  
1   Cluster 2 (47815)  
2   Cluster 3 (76962)  
3  Cluster 4 (106109)  
4  Cluster 5 (135256)  
5  Cluster 6 (164403)  
6  Cluster 7 (193550)  
7  Cluster 8 (222697)  
         Wards  Total communities  Population  Cumulative frequency  \
0          Owu     